# Performance of QTables

- Easier to manipulate columns with units

In [61]:
import astropy.units as u
from astropy.table import Table, QTable

Example with `Table`

In [62]:
t = Table()
t['time'] = [10, 20, 30] *u.min
t['speed'] = [5, 10 , 20] * u.m/u.s
print(t)

time speed
min  m / s
---- -----
10.0   5.0
20.0  10.0
30.0  20.0


In [63]:
t['time'].unit #works ok

Unit("min")

In [64]:
(t['time'] ** 2).unit #wrong result

Unit("min")

In [65]:
t['time'].unit**2 # Of course, this is still correct

Unit("min2")

Example with `QTable`

In [66]:
qt = QTable()
qt['time'] = [10, 20, 30] *u.min
qt['speed'] = [5, 10 , 20] * u.m/u.s
print(qt)

time speed
min  m / s
---- -----
10.0   5.0
20.0  10.0
30.0  20.0


In [67]:
type(qt['time']), type(t['time'])

(astropy.units.quantity.Quantity, astropy.table.column.Column)

In [68]:
qt['time'][1], t['time'][1], t['time'].quantity[1]

(<Quantity 20. min>, 20.0, <Quantity 20. min>)

In [69]:
qt['time'].unit #works ok

Unit("min")

In [70]:
(qt['time'] ** 2).unit #correct result

Unit("min2")

In [71]:
# Easy to convert 
qt2 = QTable(t)
(qt2['time']**2).unit

Unit("min2")

In [72]:
t2 = Table(qt)
(t2['time']**2).unit

Unit("min")

In [73]:
t.add_row([60*u.s, 5*u.m/u.s]) #Silently does the wrong thing
print(t) 

time speed
min  m / s
---- -----
10.0   5.0
20.0  10.0
30.0  20.0
60.0   5.0


In [76]:
qt.add_row([60*u.s, 5*u.m/u.s]) #Does the correct thing
print(qt)

time speed
min  m / s
---- -----
10.0   5.0
20.0  10.0
30.0  20.0
 1.0   5.0
 1.0   5.0


In [75]:
#same memory usage?
import sys
sys.getsizeof(t), sys.getsizeof(qt) 

(56, 56)

Application to `gammapy`

- Simplify classes eg: `LightCurve`, `FluxPoints`

eg: `e_min = self.table["e_min"].quantity` is not necessary

In [77]:
from astropy.coordinates import SkyCoord

In [78]:
t['sk'] = [SkyCoord(0,0, unit='deg'), 
           SkyCoord(0,1, unit='deg'),
           SkyCoord(1,0, unit='deg')]

ValueError: Inconsistent data column lengths

In [82]:
qt['sk'] = [SkyCoord(0,0, unit='deg'), 
           SkyCoord(0,1, unit='deg'),
           SkyCoord(1,0, unit='deg'),
            SkyCoord(1,0, unit='deg'),
            SkyCoord(1,0, unit='deg'),
           ]

In [83]:
qt.add_row([60*u.s, 5*u.m/u.s, SkyCoord(1,1, unit='deg')])

ValueError: Unable to insert row because of exception in column 'sk':
'SkyCoord' object has no attribute 'insert'

In [28]:
type(t)

astropy.table.table.Table

In [84]:
qttt = QTable([{'a': 1 * u.m, 'b': 2},
             {'a': 2 * u.m, 'b': 3}]) 

In [85]:
qttt

a,b
m,
float64,int64
1.0,2
2.0,3


In [86]:
qt = QTable(rows=[[1 * u.m, 2],
       [2 * u.m, 3]])  

In [87]:
qt

col0,col1
m,
float64,int64
1.0,2
2.0,3
